# Bulk Standard Geography
A notebook to enrich **[standard geography](https://developers.arcgis.com/rest/geoenrichment/api-reference/standard-geography-query.htm) ids**, such as `census block groups`, with their geometries and publish as a geographic layer. This uses the [`standard_geography_query`](https://esri.github.io/arcgis-python-api/apidoc/html/arcgis.geoenrichment.html#standard-geography-query) function and there's a blog about these queries [here](https://blogs.esri.com/esri/arcgis/2017/07/10/best-practices-how-to-query-standard-geographies-branches/).

In [ ]:
# common imports
import pandas as pd
from arcgis.gis import GIS
from arcgis.geoenrichment import standard_geography_query

***Note**, if you are unable to import local `partnerutils`, **copy the following functions** from [`cool_utils`](https://github.com/mpayson/esri-partner-tools/blob/master/partnerutils/cool_utils.py)

In [ ]:
from partnerutils.cool_utils import chunk

## User Input
* **GIS**: Your GIS instance, parameter information [here](https://developers.arcgis.com/python/guide/using-the-gis/). Not specifying a password creates a password prompt

In [ ]:
gis = GIS(username="mpayson_startups")

* **csv_path**: Path to the `csv` with geography IDs
* **lyr_title**: Title for output hosted layer
* **layers**: The standard geography layers whose geometry you want to fetch. Here's the [US list](http://geoenrich.arcgis.com/arcgis/rest/services/World/GeoenrichmentServer/Geoenrichment/StandardGeographyLevels/US/census?f=pjson) of layers.
* **f_std_id**: The `csv` field with the ids corresponding to a standard geography layer

In [ ]:
csv_path = '../sample_data/sample_census_tract_geoid.csv'
lyr_title = 'test out'
layers = ['US.Tracts']
f_std_id = 'census_tract_geoid'

## Execution

In [ ]:
# read df
df = pd.read_csv(csv_path, dtype={f_std_id: 'object'})

In [ ]:
# get and concat geometries, chunking because of request time-outs
ids = [i for i in df[f_std_id]]
gdf = pd.concat((standard_geography_query(layers=layers, ids=c, return_geometry=True) for c in chunk(ids)), ignore_index=True)

In [ ]:
# merge geometries
sdf = pd.merge(gdf, df, left_on='AreaID', right_on=f_std_id)

In [ ]:
# Send it!
sdf.spatial.to_featurelayer(lyr_title, tags="esri_partner_tools_sample")

## Clean up

In [ ]:
delete_items = gis.content.search("tags:esri_partner_tools_sample")
gis.content.delete_items(delete_items)